In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from import_pkg import eigenform

In [31]:
import pickle
import numpy as np
from itertools import chain
from matplotlib import pyplot as plt
from tqdm.auto import tqdm

In [4]:
norm_space_path = eigenform.get_data_dir()/"norm/norm_space.pkl"
with norm_space_path.open("rb") as fin:
    nspace = pickle.load(fin)

In [10]:
w2_space_path = eigenform.get_data_dir()/"norm/word2_space.pkl"
with w2_space_path.open("rb") as fin:
    w2_space = pickle.load(fin)    
    w2_space.cv_dist = None

In [6]:
w2_dists = w2_space.cv_dist.flatten()
print(np.quantile(w2_dists, [0.9, 0.95, 0.975, 0.99, 0.995, 0.999]))
print(len(w2_space.stoi)*(1-0.995))

[0.14136364 0.17130423 0.19940572 0.23809126 0.27557146 0.53427616]
98.70500000000008


In [12]:
import pandas as pd
non_word_ldt = pd.read_excel(eigenform.get_data_dir()/"Tse-2017-Chinese-lexicon-project.xlsx", sheet_name="Nonword")

In [19]:
with (eigenform.get_data_dir()/"asbc5_characters.pkl").open("rb") as fin:
    cfreq = pickle.load(fin)    

In [24]:
nspace.neighbors("胡", threshold=0.34)

['胡', '朗', '朋', '挈', '掌', '朝', '旂', '朔', '罈']

In [29]:
(non_word_ldt.Nonword_Trad.str.len() == 2).all()

True

In [39]:
c1_freqs = []
c2_freqs = []
c1_neigh_counts = []
c2_neigh_counts = []
nw_neigh_counts = []
for nw in tqdm(non_word_ldt.Nonword_Trad.tolist()):    
    c1 = nw[0]; c2 = nw[1]
    try:
        c1_freq = cfreq.get(c1, 1)
        c2_freq = cfreq.get(c2, 1)
        c1_nNeigh = len(nspace.neighbors(c1, threshold=0.34))
        c2_nNeigh = len(nspace.neighbors(c2, threshold=0.34))
        nw_vec = np.concatenate([nspace.get_vector(nw[0]), nspace.get_vector(nw[1])], axis=0)    
        nw_nNeigh = len(w2_space.neighbors_vector(nw_vec, 0.28))
    except:
        c1_freq = None; c2_freq = None;
        c1_nNeigh = None; c2_nNeigh = None;
        nw_nNeigh = None;
    c1_freqs.append(c1_freq)
    c2_freqs.append(c2_freq)
    c1_neigh_counts.append(c1_nNeigh)
    c2_neigh_counts.append(c2_nNeigh)
    nw_neigh_counts.append(nw_nNeigh)
    if len(c1_freqs)> 10:break

In [42]:
space_nw = non_word_ldt.iloc[:len(c1_freqs), :].assign(
    c1_logfreq=np.log(c1_freqs), c2_logfreq = np.log(c2_freqs),
    c1_nNeigh=c1_neigh_counts, c2_nNeigh=c2_neigh_counts,
    nw_nNeigh=nw_neigh_counts
)
space_nw.dropna(inplace=True)

In [43]:
space_nw.head()

,Nonword_Trad,Nonword_Sim,Ntrials,Acc,RT,RT-SE,RT-SD,zRT,c1_freq,c2_freq,c1_nNeigh,c2_nNeigh,nw_nNeigh
0,一化,一化,30,0.94,687.27,34.82,190.70,-0.18,12.243359,10.074327,8,24,485
1,一尤,一尤,30,0.91,677.83,29.39,160.96,-0.16,12.243359,8.091933,8,27,624
2,一氏,一氏,28,0.85,702.71,24.07,127.34,-0.05,12.243359,6.960348,8,24,461
3,一西,一西,33,1.00,617.42,31.78,182.57,-0.64,12.243359,9.457825,8,12,414
4,一似,一似,29,0.88,726.38,33.23,178.96,-0.05,12.243359,8.704502,8,6,287


In [44]:
space_nw.to_csv(eigenform.get_data_dir()/"norm/norm_space_ldt.csv")

In [46]:
np.log(c1_freqs)

array([12.2433588, 12.2433588, 12.2433588, 12.2433588, 12.2433588,
       12.2433588, 12.2433588, 12.2433588, 12.2433588, 12.2433588,
       12.2433588])

In [48]:
np.log([1,None])

TypeError: loop of ufunc does not support argument 0 of type int which has no callable log method